In [4]:
#CALCULATING ANGLES
def calculate_angle(a,b,c):
    a=np.array(a) #first
    b=np.array(b) #mid
    c=np.array(c) #end


    radians = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle >180.0:
        angle=360-angle

    return angle

In [ ]:
import cv2
import mediapipe as mp
import numpy as np 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

counter =0
stage=None


#setup
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose :

    while cap.isOpened():
        ret,frame = cap.read()

        #detect stuff and render

        #RECOLOR IMAGE TO RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #MAKE DETECTION
        results = pose.process(image)

        #RECOLOR IMAGE TO BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


        #extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            #GET COORDINATES
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow =  [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            #CALCULATE ANGLE
            angle=calculate_angle(shoulder,elbow,wrist)

            #visualize

            h, w, _ = image.shape
            cv2.putText(image,str(angle),
                        tuple(np.multiply(elbow,[w, h]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA
                              )
            
            #curl counter

            if angle > 160:
                stage="down"
            if angle<30 and stage =="down":
                stage="up"
                counter+=1
                print(counter)

        except:
            pass

        #render curl counter
        #setup status box

        cv2.rectangle(image,(0,0),(225,73),(245,117,16,-1))

        #rep data
        cv2.putText(image,'REPS',(15,12),
                    cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,str(counter),
                    (10,60),
                    cv2.FONT_HERSHEY_SCRIPT_COMPLEX,2,(255,255,255),2,cv2.LINE_AA)
        
        
        #STAGE data
        cv2.putText(image,'STAGE',(65,12),
                    cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,stage,
                    (60,60),
                    cv2.FONT_HERSHEY_SCRIPT_COMPLEX,2,(255,255,255),2,cv2.LINE_AA)
        
        


        #RENDER DETECTIONS

        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS)



        cv2.imshow("mediapipe feed",image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()




c:\Users\DARSHITA\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1
2
